In [ ]:
import cv2
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
from collections import deque
import json
import requests
import random

# Jetson Nano 카메라 파이프라인
pipeline = (
    "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=640, height=480, format=NV12, framerate=30/1 ! "
    "nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
)

cap = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)

if not cap.isOpened():
    print("Camera Open Error")
    exit()

video_widget = widgets.Image(format='jpeg', layout=widgets.Layout(width='640px', height='480px'))
display(video_widget)

def frame_to_bytes(frame):
    _, buf = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 40])
    return buf.tobytes()

# ArUco 설정
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
aruco_params = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_params)

# 포인트-섹터 매핑
marker_to_point = {
    1: "alpha", 2: "sector 1", 3: "sector 2", 4: "sector 3", 5: "sector 4",
    6: "sector 5", 7: "sector 6", 8: "bravo", 9: "sector 7", 10: "charlie",
    11: "sector 8", 12: "sector 9", 13: "finish"
}

visited_points = set()  # 이미 지나간 포인트를 추적

# 객체 감지: 객체 종류와 개수 (랜덤)
def generate_detected_objects():
    object_types = ['hazmat', 'enemy', 'tank', 'mortar', 'box', 'missile']
    num_objects = random.randint(1, 3)  # 각 객체는 1에서 3개 사이로 랜덤으로 생성
    return {obj: random.randint(1, 3) for obj in random.sample(object_types, num_objects)}

# 서버로 전송하는 함수
def send_dashboard(mission_code, fire_buildings, points, detection):
    data = {
        "mission_code": mission_code,
        "fire_buildings": fire_buildings,  # 특정 sector에 대한 fire_buildings 정보 추가 가능
        "points": points,  # 지나간 포인트들
        "detection": detection  # 감지된 객체 정보
    }
    try:
        json_content = json.dumps(data, indent=2, ensure_ascii=False)
        files = {'file': ('mission.json', json_content, 'application/json')}
        response = requests.post('http://58.229.150.23:5000/dashboard_json', files=files, timeout=10)
        print("서버 응답:", response.text)
    except Exception as e:
        print("서버 전송 실패:", e)

# 로그 저장용 deque
log_buffer = deque(maxlen=5)

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("[WARN] 프레임 읽기 실패")
            break

        frame = cv2.flip(frame, -1)
        corners, ids, _ = detector.detectMarkers(frame)

        if ids is not None:
            cv2.aruco.drawDetectedMarkers(frame, corners, ids)
            detected_ids = [int(i[0]) for i in ids]
            current_time = time.strftime("%H:%M:%S")
            log_buffer.append(f"[{current_time}] 감지된 ArUco 마커: {detected_ids}")

            # 포인트가 처음 감지되면 서버로 전송
            points_to_send = []
            detection = generate_detected_objects()  # 객체 감지 데이터를 생성

            for marker_id in detected_ids:
                if marker_id in marker_to_point and marker_to_point[marker_id] not in visited_points:
                    point = marker_to_point[marker_id]
                    visited_points.add(point)
                    points_to_send.append(point)

            if points_to_send:
                print(f"전송할 포인트: {points_to_send}")
                send_dashboard("####", ["sector1", "sector3"], points_to_send, detection)

            clear_output(wait=True)
            display(video_widget)
            for line in log_buffer:
                print(line)

        video_widget.value = frame_to_bytes(frame)

except KeyboardInterrupt:
    print("\n[EXIT] 카메라 스트림 종료")

finally:
    cap.release()
    print("[INFO] 리소스 해제 완료")
